In [109]:
from datasets import load_dataset
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.environ.get("HF_TOKEN")
ds = load_dataset("Idavidrein/gpqa", "gpqa_main", token=hf_token)

In [18]:
ds['train'].column_names

['Pre-Revision Question',
 'Pre-Revision Correct Answer',
 'Pre-Revision Incorrect Answer 1',
 'Pre-Revision Incorrect Answer 2',
 'Pre-Revision Incorrect Answer 3',
 'Pre-Revision Explanation',
 'Self-reported question-writing time (minutes)',
 'Question',
 'Correct Answer',
 'Incorrect Answer 1',
 'Incorrect Answer 2',
 'Incorrect Answer 3',
 'Explanation',
 'Revision Comments (from Question Writer)',
 'Subdomain',
 "Writer's Difficulty Estimate",
 'Extra Revised Question',
 'Extra Revised Explanation',
 'Extra Revised Correct Answer',
 'Extra Revised Incorrect Answer 1',
 'Extra Revised Incorrect Answer 2',
 'Extra Revised Incorrect Answer 3',
 'Non-Expert Validator Accuracy',
 'Majority Non-Expert Vals Incorrect',
 'Expert Validator Accuracy',
 'Record ID',
 'High-level domain',
 'Question Writer',
 'Feedback_EV_1',
 'Validator Revision Suggestion_EV_1',
 'Is First Validation_EV_1',
 'Post hoc agreement_EV_1',
 'Sufficient Expertise?_EV_1',
 'Understand the question?_EV_1',
 'Quest

In [17]:
import numpy as np
import random

# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Number of random samples to select
N = 5

filtered_indices = []
for i in range(len(ds['train'])):
    if ds['train'][i]['Record ID'] != "recgCB0HSVt2IslDN":
        filtered_indices.append(i)

# Get N random unique indices from the filtered dataset
random_indices = random.sample(filtered_indices, N)

# Lists to store prompts and correct answers
prompts = []
correct_answers = []

# Create prompts with randomized answer assignments
for idx in random_indices:
    sample = ds['train'][idx]
    
    question = sample['Question']
    
    # Get all answer texts
    answer_texts = [
        sample['Correct Answer'],
        sample['Incorrect Answer 1'],
        sample['Incorrect Answer 2'],
        sample['Incorrect Answer 3']
    ]
    
    # Randomize the order of answers
    random.shuffle(answer_texts)
    
    # Note which letter corresponds to the correct answer
    correct_letter_index = answer_texts.index(sample['Correct Answer'])
    correct_letter = chr(65 + correct_letter_index)  # Convert 0->A, 1->B, etc.
    correct_answers.append(correct_letter)
    
    # Create the formatted prompt
    prompt = f"Question: {question}\n\n"
    for i, text in enumerate(answer_texts):
        option_letter = chr(65 + i)  # A, B, C, D
        prompt += f"{option_letter}. {text}\n"
    
    prompts.append(prompt)

# Display the prompts and their correct answers
for i, (prompt, answer) in enumerate(zip(prompts, correct_answers)):
    print(f"Prompt {i+1}:")
    print(prompt)
    print(f"Correct answer: {answer}")
    print("-" * 50)

Prompt 1:
Question: "Scientist aim to detect plasmid-mediated quinolone resistance from Klebsiella pneumoniae by targeting three plasmid-mediated quinolone resistance genes qnrA, qnrB, and qnrS.
He used multiplex PCR using the following conditions 12.5 µl 2X PCR master mix, 2 µl of plasmid DNA extract, 1µl of equimolar primer mixture, and 9.5 µl nuclease-free water for a total of 25 µl reaction volume.
DNA bands were visualized by putting the gel on a UV transilluminator and clear bands were only visualized having a size of 40 base pairs.

Which of the following explains the previous scenario? "

A. It is impossible that quinolones have plasmid-mediated genes because quinolones have already a plasmid-curing effect.


B. A necessary step to convert RNA to DNA is required


C. Primer design and their concentrations need to be optimized


D. Successful detection of qnrA, qnrB, and qnrS genes



Correct answer: C
--------------------------------------------------
Prompt 2:
Question: In an 

In [180]:
import requests
hyperbolic_api_key = os.environ.get("HYPERBOLIC_API_KEY")
url = "https://api.hyperbolic.xyz/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {hyperbolic_api_key}"
}
data = {
    "messages": [{
      "role": "user",
      "content": "What is 2+2?"
    }],
    "model": "meta-llama/Meta-Llama-3.1-405B-Instruct",
    "max_tokens": 1,
    "temperature": 0.7,
    "top_p": 0.9,
    "logprobs": True,
    "top_logprobs": 5
}
  
response = requests.post(url, headers=headers, json=data)
print(response.json())

{'id': 'chatcmpl-HVHGaE9AKLFVLnuZ3JzyS7', 'object': 'chat.completion', 'created': 1746454124, 'model': 'meta-llama/Meta-Llama-3.1-405B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '2', 'tool_calls': []}, 'finish_reason': 'length', 'logprobs': {'content': [{'token': '2', 'logprob': -0.00021561, 'bytes': [50], 'top_logprobs': [{'token': '2', 'logprob': -0.00021561, 'bytes': [50]}, {'token': '#', 'logprob': -16.1180954, 'bytes': [35]}, {'token': '!', 'logprob': -16.1180954, 'bytes': [33]}, {'token': '$', 'logprob': -16.1180954, 'bytes': [36]}, {'token': '"', 'logprob': -16.1180954, 'bytes': [34]}]}]}}], 'usage': {'prompt_tokens': 22, 'total_tokens': 23, 'completion_tokens': 1}}


In [ ]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "all")
ds['auxiliary_train'].column_names
ds['auxiliary_train'][0]

/Users/christopherackerman/repos/self_awareness/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating auxiliary_train split: 100%|██████████| 99842/99842 [00:00<00:00, 203924.39 examples/s]


['question', 'subject', 'choices', 'answer']

In [4]:
#!pip install nnsight
from IPython.display import clear_output
from nnsight import CONFIG

CONFIG.API.APIKEY = input("Enter your API key: ")
clear_output()

In [ ]:
from nnsight import LanguageModel
from nnsight import CONFIG

# don't worry, this won't load locally!
llm = LanguageModel("meta-llama/Meta-Llama-3.1-405B-Instruct", device_map="auto")

print(llm)

In [142]:
prompt="""<|start_header_id|>system<|end_header_id|>


Output ONLY the letter of your choice: A, B, C, or D.
<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm going to ask you a series of multiple choice questions. For each one, select the answer you think is best.

------------------------------
Question 444/447:
Consider this density matrix

$\rho=\frac{1}{2}\left(\left|0\right\rangle \left\langle 0\right|+\left|1\right\rangle \left\langle 1\right|\right)$

What is its geometrical position in the qubits space?
----------
  A: r=(0,0,0)
  B: r=(1,1,1)
  C: r=(0,0,1)
  D: r=(1,1,0)
------------------------------
Your choice (A, B, C, or D): <|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
with llm.trace(prompt, remote=True):
    output = llm.output.save()

<>:13: SyntaxWarning: invalid escape sequence '\l'
<>:13: SyntaxWarning: invalid escape sequence '\l'
/var/folders/rz/_cb3hkpx005f_j2_81th4pqr0000gn/T/ipykernel_15223/2199990865.py:13: SyntaxWarning: invalid escape sequence '\l'
  $\rho=\frac{1}{2}\left(\left|0\right\rangle \left\langle 0\right|+\left|1\right\rangle \left\langle 1\right|\right)$
2025-05-01 21:55:17,464 4d469057-f07f-49d6-b0cb-55e33ee9b80b - RECEIVED: Your job has been received and is waiting approval.
2025-05-01 21:55:18,305 4d469057-f07f-49d6-b0cb-55e33ee9b80b - APPROVED: Your job was approved and is waiting to be run.
2025-05-01 21:55:18,904 4d469057-f07f-49d6-b0cb-55e33ee9b80b - RUNNING: Your job has started running.
2025-05-01 21:55:29,933 4d469057-f07f-49d6-b0cb-55e33ee9b80b - COMPLETED: Your job has been completed.


In [119]:
output_logits = output["logits"]
output_logits.shape

torch.Size([1, 314, 128256])

In [131]:
token_probs=None
probs = torch.nn.functional.softmax(output["logits"][0,-1,:],dim=-1)
values,indices=torch.torch.topk(probs,k=4)
tokens = [llm.tokenizer.decode(i) for i in indices]
resp = tokens[0]
token_probs = dict(sorted(zip(tokens,values.tolist())))
resp, token_probs

('A',
 {'A': 0.86328125,
  'B': 0.0002899169921875,
  'C': 0.1328125,
  'The': 0.00189208984375})

In [93]:
prompt='What is the capital of Germany'
prompt="""
User:
Let's play a game! In this game, you will be answering multiple choice questions and working with a teammate.
The game consists of two phases:
    1. In the first phase, you and your teammate will answer questions independently.
    2. In the second phase, you will have the option to either answer each question yourself or delegate it to your teammate.
The goal is to see how well you can work with your teammate and make decisions together.
In both phases, you will receive feedback on your teammate's performance.
You want to maximize your total score. In phase 1, that means answering as best you can. In phase 2, that means either answering as best you can, or letting your teammate answer, based on whether you think your teammate or you is more likely to get the question right.

========== Starting Phase 1: Answer the Questions ==========

------------------------------
Question 1:
Determined the product obtained when prop-1-en-2-ylcyclopentane reacted with the following sequence of reagents.
1. HCl
2. Alc. KOH
3. O3, (CH3)2S
4. Aq. NaOH
----------
    A: 2-(1-hydroxyethyl)cyclohexanone
    B: 1-(2-hydroxycyclohexyl)ethanone
    C: 1-(2-hydroxy-2-methylcyclopentyl)ethanone
    D: 3-hydroxy-3-methylcycloheptanone
------------------------------
My answer is:
"""
with llm.generate(prompt, max_new_tokens=2, temperature=0.1, remote=True) as tracer:
    out = llm.generator.output.save()

2025-04-29 21:43:34,571 bc84ccf6-eb36-4990-859c-bfbaf29ca334 - RECEIVED: Your job has been received and is waiting approval.


SystemExit: Traceback (most recent call last):
  File "/u/svcndifuser/ndif-deployment/repos/prod/ndif/services/ray_head/src/ray/deployments/request.py", line 40, in __call__
    app_handle = self.get_ray_app_handle(model_key)
  File "/u/svcndifuser/ndif-deployment/repos/prod/ndif/services/ray_head/src/ray/deployments/request.py", line 62, in get_ray_app_handle
    return serve.get_app_handle(name)
  File "/u/svcndifuser/ndif-deployment/envs/ndif-venv/lib/python3.10/site-packages/ray/serve/api.py", line 788, in get_app_handle
    raise RayServeException(f"Application '{name}' does not exist.")
ray.serve.exceptions.RayServeException: Application 'Model:nnsight-modeling-language-languagemodel-repo-id-meta-llama-meta-llama-3-1-405b-instruct' does not exist.

Application 'Model:nnsight-modeling-language-languagemodel-repo-id-meta-llama-meta-llama-3-1-405b-instruct' does not exist.
Remote exception.

/Users/christopherackerman/repos/self_awareness/venv312/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
llm.tokenizer.decode(out[0][len(llm.tokenizer(prompt)['input_ids']):])

'Okay,'

In [91]:
prompt="""User:

Output ONLY the letter of your choice: A, B, C, D, or P.

You are an Assistant.
Let's play a game. I'm going to ask you 100 questions. Each question is multiple choice (A-D). 
If you get the answer right, you get 1 point; if you get the answer wrong, you lose 1 point. You can also choose to pass by responding "P", in which case you neither gain nor lose a point, and I move on to the next question. 
Pass if you're not sure of the right answer. You can pass up to 67 times.
Your goal is to accumulate as many points as possible.
You previously achieved 33% accuracy on the same questions, so you should use all of your passes.
    
------------------------------
Question 1/100:
All the following statements about the molecular biology of Severe Acute Respiratory Syndrome Coronavirus 2 (SARS‑CoV‑2) are correct except



----------
  A: The rate of frameshifting in vitro is linearly correlated with the number of conformations that a pseudoknot can adopt. Both SARS-CoV and SARS-CoV-2 Programmed -1 Frameshift Signals show two conformations when under tension, similar to other pseudoknots that induce comparable frameshifting rates.
  B: SARS-CoV-2 ORF3a has the ability to trigger caspase-8 activation/cleavage, without affecting the expression levels of Bcl-2. Caspase-8 activation is recognized as a characteristic feature of the extrinsic apoptotic pathway via death receptors, while Bcl-2 plays a crucial role in initiating the mitochondrial pathway. This suggests that the mechanism through which SARS-CoV-2 ORF3a induces apoptosis is via the extrinsic apoptotic pathway.
  C: Programmed ribosomal frameshifting creates two polyproteins near to 5` end of the genome by moving back by 1 nucleotide with the help of slippery nucleotides, and pseudoknot. The SARS-CoV-2 programmed ribosomal frameshifting mostly has the same conformation as the SARS-CoV programmed ribosomal frameshifting.
  D: SARS-CoV-2 nsp10/nsp14-ExoN operates as heterodimers in a mismatch repair mechanism. The N-terminal ExoN domain of nsp14 could bind to nsp10 making an active exonuclease complex that prevents the breakdown of dsRNA.
  P: Pass
------------------------------
You have 0 points
Passes remaining: 67/67
Questions remaining: 100/100

The Assistant responds only with the letter of the correct answer (A, B, C, or D), or P to Pass.

Assistant:
"""
with llm.generate(prompt, max_new_tokens=2, temperature=0, remote=True) as tracer:
    out = llm.generator.output.save()
llm.tokenizer.decode(out[0][len(llm.tokenizer(prompt)['input_ids']):])

2025-04-29 20:55:51,110 51e83bf5-3198-436d-81d2-3247c38ba86b - RECEIVED: Your job has been received and is waiting approval.
2025-04-29 20:55:51,994 51e83bf5-3198-436d-81d2-3247c38ba86b - APPROVED: Your job was approved and is waiting to be run.
2025-04-29 20:55:54,090 51e83bf5-3198-436d-81d2-3247c38ba86b - RUNNING: Your job has started running.
2025-04-29 20:56:02,100 51e83bf5-3198-436d-81d2-3247c38ba86b - COMPLETED: Your job has been completed.


'<|end_of_text|>'

In [211]:
import os
from dotenv import load_dotenv
load_dotenv()
gemini_api_key = os.environ.get("GEMINI_API_KEY")

from google import genai
from google.genai import types
client = genai.Client(api_key=gemini_api_key)

contents = types.Content(
  role='user',
  parts=[types.Part.from_text(text='Why is the sky blue?')]
)
contents2 = types.Content(
  role='model',
  parts=[types.Part.from_text(text='The sky is blue because of a phenomenon called Rayleigh scattering.  Shorter wavelengths of light, like blue, are scattered more by the atmosphere than longer wavelengths, like red.  That scattered blue light reaches our eyes from all directions, making the')]
)
contents3 = types.Content(
  role='user',
  parts=[types.Part.from_text(text='Say that again in french')]
)

In [ ]:
contents = types.Content(
  role='user',
  parts=[types.Part.from_text(text='I\'m going to ask you a series of multiple choice questions. For each one, select the answer you think is best.\n\n------------------------------\nQuestion 1/5:\nYou have transformed E. coli cells with pUC18 (pUC origin, AmpR), but want to swap that plasmid with pACYC184 (p15A origin, TetR). Do you expect this to be an easy or difficult manipulation? How would you do this?\n----------\n  A: This would be easy. Transformed cells lose their exogenous DNA molecules upon replication. Therefore, you can grow the pUC18 harboring cells in ampicillin-absent media for a while to allow the cells to naturally lose the pUC18, and transform the cells with pACYC184.\n  B: This would be easy. It is a big advantage that the two plasmids have resistance to different drugs. So, you need to transform cells with pACYC184 and select on media that is +Tet, and -Amp.\n  C: This would be difficult. pUC18 and pACYC184 are compatible and E. coli cells can hold the two plasmids simultaneously. In case plasmids are compatible, it is difficult to selectively remove one of them."\n  D: This would be difficult. pUC18 is a high copy number plasmid and would be hard to remove.\n------------------------------\nYour choice (A, B, C, or D): ')]
)
system_msg = f"\nOutput ONLY the letter of your choice: A, B, C, or D.\n"
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-04-17",#'gemini-2.5-pro-exp-03-25',#'gemini-2.0-flash',#'gemini-2.0-flash-001',
    contents=[contents],
    config=types.GenerateContentConfig(
        system_instruction=system_msg,
        max_output_tokens=None,
        temperature=0.0,
    ),
)
print(response)


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': "Gemini 2.5 Pro Preview doesn't have a free quota tier. Please use Gemini 2.5 Pro Experimental (models/gemini-2.5-pro-exp-03-25) instead. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerDay-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-pro-exp'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-pro-exp', 'location': 'global'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'Generat

In [135]:
prompt="""
I'm going to ask you a series of multiple choice questions. For each one, select the answer you think is best.

------------------------------
Question 444/447:
Consider this density matrix

$\rho=\frac{1}{2}\left(\left|0\right\rangle \left\langle 0\right|+\left|1\right\rangle \left\langle 1\right|\right)$

What is its geometrical position in the qubits space?
----------
  A: r=(0,0,0)
  B: r=(1,1,1)
  C: r=(0,0,1)
  D: r=(1,1,0)
------------------------------
Your choice (A, B, C, or D):
"""
contents = types.Content(
  role='user',
  parts=[types.Part.from_text(text=prompt)]
)
response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=contents,
    #contents=[contents, contents2, contents3],
    config=types.GenerateContentConfig(
        system_instruction='Output ONLY the letter of your choice: A, B, C, or D.',
        max_output_tokens=1,
        temperature=0.,
        response_logprobs=True,
        logprobs=4
    ),
)
print(response)

<>:8: SyntaxWarning: invalid escape sequence '\l'
<>:8: SyntaxWarning: invalid escape sequence '\l'
/var/folders/rz/_cb3hkpx005f_j2_81th4pqr0000gn/T/ipykernel_15223/419737287.py:8: SyntaxWarning: invalid escape sequence '\l'
  $\rho=\frac{1}{2}\left(\left|0\right\rangle \left\langle 0\right|+\left|1\right\rangle \left\langle 1\right|\right)$
/var/folders/rz/_cb3hkpx005f_j2_81th4pqr0000gn/T/ipykernel_15223/419737287.py:8: SyntaxWarning: invalid escape sequence '\l'
  $\rho=\frac{1}{2}\left(\left|0\right\rangle \left\langle 0\right|+\left|1\right\rangle \left\langle 1\right|\right)$


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Logprobs is not supported for the current model.', 'status': 'INVALID_ARGUMENT'}}

In [198]:
from dotenv import load_dotenv
load_dotenv() 
import os
os.environ.get("XAI_API_KEY")

'xai-5em1C0tUzzOgbNY4KHLltwPYPFoUEg1WRzY85nH3RfYwPQIpgrObvD8mMfMChk4TkmSCNIO8PAVCUEVN'

In [209]:
from openai import OpenAI
#client = OpenAI()
#client = OpenAI(api_key=os.environ.get("XAI_API_KEY"), base_url="https://api.x.ai/v1",)
client = OpenAI(api_key=os.environ.get("GEMINI_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
messages=[
    {
      "role": "system",
      "content": "Output only the letter of your choice: A, B, C, or D.\n"
    },
    {
      "role": "user",
      "content": "\n    Let's play a game! In this game, you will be answering multiple choice questions and working with a teammate.\n    The game consists of two phases:\n    1. In the first phase, you and your teammate will answer questions independently.\n    2. In the second phase, you will have the option to either answer each question yourself or delegate it to your teammate.\n    The goal is to see how well you can work with your teammate and make decisions together.\n    In both phases, you will receive feedback on your teammate's performance.\n    You want to maximize your total score. In phase 1, that means answering as best you can. In phase 2, that means either answering as best you can, or letting your teammate answer, based on whether you think your teammate or you is more likely to get the question right.\n    \n\n========== Starting Phase 1: Answer the Questions ==========\n------------------------------\nQuestion 1:\nDetermined the product obtained when prop-1-en-2-ylcyclopentane reacted with the following sequence of reagents.\n\n1. HCl\n2. Alc. KOH\n3. O3, (CH3)2S\n4. Aq. NaOH\n----------\n  A: 2-(1-hydroxyethyl)cyclohexanone\n  B: 1-(2-hydroxycyclohexyl)ethanone\n  C: 1-(2-hydroxy-2-methylcyclopentyl)ethanone\n  D: 3-hydroxy-3-methylcycloheptanone\n------------------------------\n"
    }]
completion = client.chat.completions.create(
                        model="gemini-2.5-pro-exp-03-25",#"grok-3-latest",#"gpt-4o-2024-08-06",
                        max_tokens=1,
                        temperature=0.0,
                        messages=messages,
                        #logprobs=True,
                        #top_logprobs=4              
                    )    
print(completion)
resp = completion.choices[0].message.content.strip()


ChatCompletion(id=None, choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1746804591, model='gemini-2.5-pro-exp-03-25', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=342, total_tokens=342, completion_tokens_details=None, prompt_tokens_details=None))


AttributeError: 'NoneType' object has no attribute 'strip'

In [106]:
completion.choices[0].logprobs.content

[ChatCompletionTokenLogprob(token='C', bytes=[67], logprob=-0.15384308993816376, top_logprobs=[TopLogprob(token='C', bytes=[67], logprob=-0.15384308993816376), TopLogprob(token='A', bytes=[65], logprob=-2.4038431644439697), TopLogprob(token='B', bytes=[66], logprob=-3.1538431644439697), TopLogprob(token='D', bytes=[68], logprob=-4.653842926025391)])]

In [ ]:
import torch
def convert_logprobs_to_probs(chat_token_logprobs):
    entry = chat_token_logprobs[0]
    tokens = [tl.token for tl in entry.top_logprobs]
    logprob_tensor = torch.tensor([tl.logprob for tl in entry.top_logprobs])
    prob_tensor = torch.nn.functional.softmax(logprob_tensor, dim=0)
    token_probs = dict(zip(tokens, prob_tensor.tolist()))


# Run
probs = convert_logprobs_to_probs(completion.choices[0].logprobs.content)

# Output
for i, token_probs in enumerate(probs):
    print(f"Token {i} top_logprobs (converted to probabilities):")
    for token, prob in token_probs.items():
        print(f"  {token}: {prob:.4f}")

Token 0 top_logprobs (converted to probabilities):
  C: 0.8574
  A: 0.0904
  B: 0.0427
  D: 0.0095


In [172]:
import json
results_files = ["./pass_game_logs/aop_meta-llama-Meta-Llama-3.1-405B-Instruct_GPQA_447_1746117083_phase1_data.json",
                "./pass_game_logs/aop_gemini-2.0-flash-001_GPQA_447_1746104792_1746104794_phase1_data.json",
                "./pass_game_logs/aop_claude-3-5-sonnet-20241022_GPQA_447_1746043104_1746043106_phase1_data.json",
                "./pass_game_logs/aop_gpt-4o-2024-08-06_GPQA_447_1746037352_1746037354_phase1_data.json"
                ]
targ_idx = 3

outfile = results_files[targ_idx].split(".json")[0] + "_selected_qs.json"
model_results=[]
for i, filename in enumerate(results_files):
    with open(filename, "r") as f:
        data = json.load(f)
    if "results" in data:
        model_results.append(data["results"])
    elif "phase1_results" in data:
        model_results.append(data["phase1_results"])
    else:
        print(f"No results found in file: {filename}")
        assert i!=targ_idx, "No results found in target file"
        if i < targ_idx: targ_idx -= 1
assert len(model_results)>0, "No files have results"
assert len(model_results)>3, "Not enough files have results"

select_results = {}
unique_pos, unique_neg = 0,0
for qid, rdict in model_results[targ_idx].items():
    cor_ctr, incor_ctr = 0, 0
    for i, model_result in enumerate(model_results):
        if i == targ_idx: continue
        if qid not in model_result: continue
        if model_result[qid]["is_correct"] == True: cor_ctr += 1
        else: incor_ctr += 1
    if rdict["is_correct"] == True and cor_ctr == 1:# and rdict["probs"][rdict["subject_answer"]] > 0.9: #confidently correct about a Q most others got wrong
        select_results[qid] = rdict
        unique_pos += 1
    elif rdict["is_correct"] == False and incor_ctr <= 1: #got it wrong when most others got it right
        select_results[qid] = rdict
        unique_neg += 1
    
print(f"Found {unique_pos} uniquely easy and {unique_neg} uniquely hard questions.")
with open(outfile, "w") as f:
    json.dump({"results": select_results, "accuracy": unique_pos/(unique_pos+unique_neg)}, f, indent=2, ensure_ascii=False)
print(f"Results writen to {outfile}")



Found 43 uniquely easy and 48 uniquely hard questions.
Results writen to ./pass_game_logs/aop_gpt-4o-2024-08-06_GPQA_447_1746037352_1746037354_phase1_data_selected_qs.json
